In [98]:
import random
import matplotlib.pyplot as plt

In [99]:
def reshuffle(deck, trash):
    random.shuffle(trash)
    if len(deck) == 0:
        deck = trash
    else:
        deck.extend(trash)
    # 捨札の初期化をする処理
    trash = []
    return deck, trash 

In [100]:
def smith(hand, deck, trash):
    if len(deck) <= 3:
        deck, trash = reshuffle(deck, trash)
    else:
        pass
    hand.extend(deck[:3])
    del deck[:3]
    return hand, deck, trash

In [101]:
def purchase_phase(hand, trash, province, s_flag, g_flag, s_thresh, g_thresh):
    money = sum([i for i in hand if str.isdigit(str(i))])
    if money >= 8 and g_flag >= g_thresh:
            trash.insert(0, 0)
            province += 1
    else:
        if money >= 6:
            trash.insert(0, 3)
            g_flag += 1
        else:
            if (money == 4 or money == 5) and s_flag < s_thresh:
                trash.insert(0, 's')
                s_flag += 1 # 以降鍛冶屋を買わない
            else:
                if money == 3 or money == 4 or money == 5:
                    trash.insert(0, 2)
                else:
                    pass
    return trash, province, s_flag, g_flag

In [102]:
def cleanup_phase(hand, deck, trash):
    trash.extend(hand)
    if len(deck) <= 4:
        # デッキの残りと捨札を混ぜて新しいデッキを作成する処理
        deck, trash = reshuffle(deck, trash)
    return deck, trash

In [106]:
def game(s_thresh, g_thresh):
    """
    入力
    - s_thresh : smith_threshold_valueの略、鍛冶屋を何枚購入するか 
    - g_thresh : glod_threshold_valueの略、8金以上でたときに何枚金貨を獲得していたら属州を購入するか
    
    出力
    - turn : 属州4枚獲得するのに経過したターン数
    """
    deck = [1, 1, 1, 1, 1, 1, 1, 0, 0, 0] #デッキの初期化
    trash = [] #捨て札の初期化
    province = 0 #属州の枚数の初期化
    s_flag = 0 # 鍛冶屋フラグの初期化
    g_flag = 0 # 金貨取得枚数の初期化
    turn = 1 # ターン数の初期化：スタート値:1
    random.shuffle(deck) #デッキのシャッフル
    while province < 4: #属州を4枚獲得したらストップする
        # 5枚引く動き
        hand = deck[:5].copy()
        # デッキから5枚削除する
        del deck[:5]
        
        # アクションフェーズ：鍛冶屋があれば使用する
        if 's' in hand:
            hand, deck, trash = smith(hand, deck, trash)
        
        # 購入フェイズ           
        trash, province, s_flag, g_flag = purchase_phase(hand, trash, province, s_flag, g_flag, s_thresh, g_thresh)
        
        # 5枚ドローを除いたクリーンアップフェイズ
        deck, trash = cleanup_phase(hand, deck, trash)

        turn += 1 #turn数の追加
    return turn

In [134]:
onlymoney = [game(0, 2) for i in range(10000)]
smith1 = [game(1, 2) for i in range(10000)]
smith2 = [game(2, 2) for i in range(10000)]
smith3 = [game(3, 2) for i in range(10000)]

In [ ]:
fig, axs = plt.subplots(4, 1, figsize=(10, 10))
axs = axs.ravel()
axs[0].hist(onlymoney, label='only-money', color='blue')
axs[0].legend(loc='upper left')
axs[1].hist(smith1, label='smith_1', color='red')
axs[1].legend(loc='upper left')
axs[2].hist(smith2, label='smith_2', color='green')
axs[2].legend(loc='upper left')
axs[3].hist(smith3, label='smith_3', color='orange')
axs[3].legend(loc='upper left')
plt.savefig('dominion.png')